<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/docs/quests/EDAs/EDA_LetalCarOfContractType.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
df_LCOCT = pd.read_csv('LetalCarOfContractType.csv')

## 제안 8: 계약 갱신율 최적화 전략
- 문제 정의: 다양한 계약 유형 및 고객 특성을 분석하여 계약 갱신율을 증가시키는 전략을 개발합니다.
- 배경: 계약 갱신율을 높이기 위한 전략을 수립함으로써 장기적인 고객 관계를 유지하고, 안정적인 수익 흐름을 확보할 수 있습니다.


### DDA

In [3]:
df_LCOCT.columns

Index(['id', 'type_of_contract', 'type_of_contract2', 'channel', 'datetime',
       'Term', 'payment_type', 'product', 'amount', 'state', 'overdue_count',
       'overdue', 'credit rating', 'bank', 'cancellation', 'age', 'Mileage'],
      dtype='object')

In [4]:
df_LCOCT[:5]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [29]:
df_LCOCT['type_of_contract'].value_counts()
# 멤버십 고객보다 렌탈 고객이 10배라는 점을 염두 -> 렌탈 비율이 많은 경우 비율을 확인해봐야함

렌탈     46481
멤버십     4819
Name: type_of_contract, dtype: int64

#### 렌탈 재계약

In [26]:
df_LCOCT_re = df_LCOCT.query('channel == "렌탈재계약"')
df_LCOCT_re[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [27]:
df_LCOCT_re['type_of_contract'].value_counts()
# 렌탈 재계약한 사람들 중 대부분은 멤버십을 이용하는 사람들임. 따라서 계약 갱신을 위해서는 멤버십 가입을 장려해야함

멤버십    2192
렌탈        1
Name: type_of_contract, dtype: int64

In [28]:
# 멤버십 고객들은 얼마나 많이 렌탈 재계약을 할까?
df_LCOCT.query('type_of_contract == "멤버십"')['channel'].value_counts()
# 렌탈재계약인 2192명 을 제외한 약 3000명의 멤버십 고객들은 신규고객임
# 그 중 하이마트를 통해 들어오는 신규고객이 다수이므로 해당 플랫폼에 대한 강화정책(????) 이 필요하지 않을까

렌탈재계약      2192
하이마트       2086
R관리방판       449
R농협인터넷몰      81
R빌리미         11
Name: channel, dtype: int64

In [ ]:
# 하이마트로 들어온 신규 고객 중 렌탈재계약을 하는 비율을 알 수 있으면 좋을 것 같음

#### 해약한 고객

In [19]:
df_LCOCT['cancellation'].value_counts()

정상    50372
해약      907
Name: cancellation, dtype: int64

In [18]:
df_LCOCT['state'].value_counts()  # 해약진행중인 경우에도 정상으로 뜨기 때문에 state를 기준

계약확정     50350
해약확정       907
기간만료        33
해약진행중       14
Name: state, dtype: int64

In [17]:
condition1 = 'state == "해약확정"'
condition2 = 'state == "해약진행중"'
df_LCOCT_cancel = df_LCOCT.query(f'{condition1} or {condition2}')

In [23]:
df_LCOCT_cancel['type_of_contract'].value_counts()
# 해약 진행한 이들 중 멤버십을 가진 이들이 더 많음. 멤버십 인원이 렌탈인원의 1/10라는 점을 감안했을 때
# 멤버십 고객 중 해약하는 비율이 높다고 판담
# 해약하는 비율이 높은 이유를 찾아보자

멤버십    563
렌탈     358
Name: type_of_contract, dtype: int64

인사이트 : 멤버십 고객이